
# -- 🚗 Used Car Price Prediction --
# 4. Model Optimization –  (RF, XGB, LGBM)

Bu notebookta 3. notebookta oluşturulan feature engineering yapısı kullanılarak
RandomForest, XGBoost ve LightGBM modelleri karşılaştırılacaktır.

- Feature set
- Train / Validation / Test ayrımı
- Preprocessing (OneHotEncoder + sayısal kolonlar)


In [1]:
# Kütüphaneler
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use("seaborn-v0_8")
pd.set_option("display.max_columns", None)


In [2]:
# Veri Seti
df = pd.read_csv("/kaggle/input/automl8888888/used_cars_dataset_v2.csv")

def clean_km(x):
    if pd.isna(x):
        return np.nan
    x = str(x).lower().replace("km", "").replace(",", "").strip()
    try:
        return float(x)
    except ValueError:
        return np.nan

def clean_price(x):
    if pd.isna(x):
        return np.nan
    x = str(x)
    x = (
        x.replace("₹", "")
         .replace(",", "")
         .replace("rs.", "")
         .replace("rs", "")
         .strip())
    try:
        return float(x)
    except ValueError:
        return np.nan

df["kmDriven_clean"] = df["kmDriven"].apply(clean_km)
df["AskPrice_clean"] = df["AskPrice"].apply(clean_price)
df = df.dropna(subset=["kmDriven_clean", "AskPrice_clean"])
df.head()

,Brand,model,Year,Age,kmDriven,Transmission,Owner,FuelType,PostedDate,AdditionInfo,AskPrice,kmDriven_clean,AskPrice_clean
0,Honda,City,2001,23,"98,000 km",Manual,second,Petrol,Nov-24,"Honda City v teck in mint condition, valid gen...","₹ 1,95,000",98000.0,195000.0
1,Toyota,Innova,2009,15,190000.0 km,Manual,second,Diesel,Jul-24,"Toyota Innova 2.5 G (Diesel) 7 Seater, 2009, D...","₹ 3,75,000",190000.0,375000.0
2,Volkswagen,VentoTest,2010,14,"77,246 km",Manual,first,Diesel,Nov-24,"Volkswagen Vento 2010-2013 Diesel Breeze, 2010...","₹ 1,84,999",77246.0,184999.0
3,Maruti Suzuki,Swift,2017,7,"83,500 km",Manual,second,Diesel,Nov-24,Maruti Suzuki Swift 2017 Diesel Good Condition,"₹ 5,65,000",83500.0,565000.0
4,Maruti Suzuki,Baleno,2019,5,"45,000 km",Automatic,first,Petrol,Nov-24,"Maruti Suzuki Baleno Alpha CVT, 2019, Petrol","₹ 6,85,000",45000.0,685000.0


In [3]:
# Modelleme için temel set

features = [
    "Brand", "model", "Year", "Age",
    "kmDriven_clean", "Transmission",
    "Owner", "FuelType"]

target = "AskPrice_clean"

df_model = df[features + [target]].copy()
df_model.head()


,Brand,model,Year,Age,kmDriven_clean,Transmission,Owner,FuelType,AskPrice_clean
0,Honda,City,2001,23,98000.0,Manual,second,Petrol,195000.0
1,Toyota,Innova,2009,15,190000.0,Manual,second,Diesel,375000.0
2,Volkswagen,VentoTest,2010,14,77246.0,Manual,first,Diesel,184999.0
3,Maruti Suzuki,Swift,2017,7,83500.0,Manual,second,Diesel,565000.0
4,Maruti Suzuki,Baleno,2019,5,45000.0,Automatic,first,Petrol,685000.0


In [4]:
# Feature Engineering

df_fe = df_model.copy()

# 1) km başına fiyat
df_fe["price_per_km"] = df_fe["AskPrice_clean"] / (df_fe["kmDriven_clean"] + 1)

# 2) yıllık km
df_fe["km_per_year"] = df_fe["kmDriven_clean"] / (df_fe["Age"] + 1)

# 3) log dönüşümleri
df_fe["log_kmDriven"] = np.log1p(df_fe["kmDriven_clean"])
df_fe["log_price"] = np.log1p(df_fe["AskPrice_clean"])

df_fe.head()


,Brand,model,Year,Age,kmDriven_clean,Transmission,Owner,FuelType,AskPrice_clean,price_per_km,km_per_year,log_kmDriven,log_price
0,Honda,City,2001,23,98000.0,Manual,second,Petrol,195000.0,1.989776,4083.333333,11.492733,12.180760
1,Toyota,Innova,2009,15,190000.0,Manual,second,Diesel,375000.0,1.973674,11875.000000,12.154785,12.834684
2,Volkswagen,VentoTest,2010,14,77246.0,Manual,first,Diesel,184999.0,2.394902,5149.733333,11.254763,12.128111
3,Maruti Suzuki,Swift,2017,7,83500.0,Manual,second,Diesel,565000.0,6.766386,10437.500000,11.332614,13.244583
4,Maruti Suzuki,Baleno,2019,5,45000.0,Automatic,first,Petrol,685000.0,15.221884,7500.000000,10.714440,13.437176


In [5]:
# Train / Validation / Test Split (%70–15–15)

X_fe = df_fe.drop("AskPrice_clean", axis=1)
y_fe = df_fe["AskPrice_clean"]

# 1) Train (%70) - Temp (%30)
X_train_full, X_temp, y_train_full, y_temp = train_test_split(
    X_fe, y_fe,
    test_size=0.30,
    random_state=42)

# 2) Temp → Validation (%15) - Test (%15)
X_val, X_test_final, y_val, y_test_final = train_test_split(
    X_temp, y_temp,
    test_size=0.50,
    random_state=42)

X_train_full.shape, X_val.shape, X_test_final.shape


((10433, 12), (2236, 12), (2236, 12))

In [6]:
# Preprocessor

cat_cols = ["Brand", "model", "Transmission", "Owner", "FuelType"]
num_cols = ["Year", "Age", "kmDriven_clean",
            "price_per_km", "km_per_year", "log_kmDriven", "log_price"]

preprocessor_fe = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
        ("num", "passthrough", num_cols)])


## 4.1 RandomForest, XGBoost, LightGBM Modellerin Kurulması


In [7]:
# Modeller (RF, XGB, LGBM)
# RandomForest
rf_model = Pipeline(steps=[
    ("preprocess", preprocessor_fe),
    ("model", RandomForestRegressor(
        n_estimators=300,
        random_state=42,
        n_jobs=-1))])

# XGBoost
xgb_model = Pipeline(steps=[
    ("preprocess", preprocessor_fe),
    ("model", XGBRegressor(
        n_estimators=300,
        learning_rate=0.1,
        max_depth=6,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        n_jobs=-1,
        tree_method="hist"))])

# LightGBM
lgbm_model = Pipeline(steps=[
    ("preprocess", preprocessor_fe),
    ("model", LGBMRegressor(
        n_estimators=300,
        learning_rate=0.1,
        random_state=42,
        n_jobs=-1))])

rf_model, xgb_model, lgbm_model


(Pipeline(steps=[('preprocess',
                  ColumnTransformer(transformers=[('cat',
                                                   OneHotEncoder(handle_unknown='ignore'),
                                                   ['Brand', 'model',
                                                    'Transmission', 'Owner',
                                                    'FuelType']),
                                                  ('num', 'passthrough',
                                                   ['Year', 'Age',
                                                    'kmDriven_clean',
                                                    'price_per_km',
                                                    'km_per_year',
                                                    'log_kmDriven',
                                                    'log_price'])])),
                 ('model',
                  RandomForestRegressor(n_estimators=300, n_jobs=-1,
                           

## 4.2 Modellerin Validation Set Üzerinde Karşılaştırılması


In [8]:
#  Validation Performans Karşılaştırması

models = {
    "RandomForest": rf_model,
    "XGBoost": xgb_model,
    "LightGBM": lgbm_model}

results = []

for name, model in models.items():
    model.fit(X_train_full, y_train_full)
    y_val_pred = model.predict(X_val)
    
    mse = mean_squared_error(y_val, y_val_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_val, y_val_pred)
    r2 = r2_score(y_val, y_val_pred)
    
    results.append({
        "Model": name,
        "MSE": mse,
        "RMSE": rmse,
        "MAE": mae,
        "R2": r2})

results_df = pd.DataFrame(results)
results_df


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000997 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1613
[LightGBM] [Info] Number of data points in the train set: 10433, number of used features: 147
[LightGBM] [Info] Start training from score 1009359.021854


,Model,MSE,RMSE,MAE,R2
0,RandomForest,1.136894e+08,10662.522984,1066.425493,0.999950
1,XGBoost,9.342390e+09,96656.041188,21257.610582,0.995905
2,LightGBM,3.332650e+10,182555.479344,18817.423581,0.985393


## 4.3 Model Optimization Özet Bulgular

- Bu notebookta 3. notebookta üretilen feature set ile
  **RandomForest, XGBoost ve LightGBM** modelleri karşılaştırıldı.

- Validation set performansına göre en yüksek R² değerini veren model, RF modelidir.
- RF modeli final modeldir.


- Bir sonraki notebookta (5. Evaluation) bu model için:
  - Test set performansı,
  - Feature importance,
  - SHAP analizleri
  yapılacaktır.
